In [ ]:
from functools import reduce

In [ ]:
import bqplot
from IPython.display import display, Markdown
import ipywidgets
import matplotlib
import numpy as np
import pandas as pd
import scipy.stats
import xarray as xr

In [ ]:
import plot
import util

In [ ]:
eigenbreasts = xr.open_dataset('../data/processed/mri-eigenbreasts.nc')

## Distribution ##

Histrograms of first PC.

In [ ]:
for var in eigenbreasts.data_vars:
    plot.hist(eigenbreasts[var][0, :], title=var)

Scatterplots of first and second PC.

In [ ]:
for var in eigenbreasts.data_vars:
    with plot.subplots() as (fig, ax):
        plot.scatter(eigenbreasts[var][0, :], eigenbreasts[var][1, :], ax=ax)
        ax.set_title(var)

Outliers per datsets, as defined by being outside two times the inter-quartile range.

In [ ]:
outliers = dict()
for var in eigenbreasts.data_vars:
    val = eigenbreasts[var][0, :]
    outliers[var] = set(eigenbreasts['case'][np.abs(val - np.median(val)) > 2*scipy.stats.iqr(val.values)].values)
all_outlier_cases = sorted(reduce(set.union, outliers.values()))
all_vars = eigenbreasts.data_vars
outlier_da = xr.DataArray(
    np.array([np.isin(all_outlier_cases, list(outliers[v])) for v in all_vars]),
    dims=['var', 'case'],
    coords={'var': all_vars, 'case': all_outlier_cases},
)
with plot.subplots() as (fig, ax):
    plot.heatmap(outlier_da, cmap='Greys', ax=ax, col_dendrogram=True, col_dist_metric='cityblock',
                zlabel='Outlier (Outside 2*IQR)')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

## Correlation between ipsi, contra, and both ##

In [ ]:
with plot.subplots(1, 3, figsize=(12, 5)) as  (fig, axs):
    ipsi_contra_cor = util.cor(
        eigenbreasts['contra_ds8'].isel(PC=slice(0, 50)).rename({'PC': 'PC_contra_ds8'}),
        eigenbreasts['ipsi_ds8'].isel(PC=slice(0, 50)).rename({'PC': 'PC_ipsi_ds8'}),
        'case',
    )
    plot.heatmap(np.abs(ipsi_contra_cor['correlation']), xticklabels='', yticklabels='',
                 zlabel="$|\\rho|$", ax=axs[0], cbar=False, zlim=(0, 1))
    ipsi_both_cor = util.cor(
        eigenbreasts['both_ds8'].isel(PC=slice(0, 50)).rename({'PC': 'PC_both_ds8'}),
        eigenbreasts['ipsi_ds8'].isel(PC=slice(0, 50)).rename({'PC': 'PC_ipsi_ds8'}),
        'case',
    )
    plot.heatmap(np.abs(ipsi_both_cor['correlation']), xticklabels='', yticklabels='',
                zlabel="$|\\rho|$", ax=axs[1], cbar=False, zlim=(0, 1))
    contra_both_cor = util.cor(
        eigenbreasts['both_ds8'].isel(PC=slice(0, 50)).rename({'PC': 'PC_both_ds8'}),
        eigenbreasts['contra_ds8'].isel(PC=slice(0, 50)).rename({'PC': 'PC_contra_ds8'}),
        'case'
    )
    plot.heatmap(np.abs(contra_both_cor['correlation']), xticklabels='', yticklabels='',
                 zlabel="$|\\rho|$", ax=axs[2], cbar=False, zlim=(0, 1))
    fig.colorbar(axs[0].findobj(matplotlib.image.AxesImage)[0], ax=list(axs),
                 orientation='horizontal', fraction=0.05, label="$|\\rho|$")
    for ax in axs:
        ax.axis('equal', adjustable='box')
        ax.set_aspect('equal', 'datalim')
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)